In [2]:
#import libraries
import numpy as np
import pandas as pd

#####read csv files#####

#specify the path
path = ('../Project_Mod3_draft/uw-madison-courses')

#grades
grades = pd.read_csv(f'{path}/grade_distributions.csv')
sections = pd.read_csv(f'{path}/sections.csv')

#instructions
teachings = pd.read_csv(f'{path}/teachings.csv')
instructors = pd.read_csv(f'{path}/instructors.csv')

#subjects
subject_mem = pd.read_csv(f'{path}/subject_memberships.csv')
subjects = pd.read_csv(f'{path}/subjects.csv')
is_stem = pd.read_csv(f'{path}/is_stem.csv',index_col=0)

# #courses
course_offer = pd.read_csv(f'{path}/course_offerings.csv')
# courses = pd.read_csv(f'{path}/courses.csv')

# #schedules and rooms
# schedules = pd.read_csv(f'{path}/schedules.csv')
# rooms = pd.read_csv(f'{path}/rooms.csv')

In [167]:
#####process grades#####

#add column with total count of grades for each course
count_cols = [x for x in grades.columns if 'count' in x]
grades['class_size'] = grades[count_cols].sum(axis=1)

#select needed columns
g = grades[['course_offering_uuid','section_number','a_count','class_size']]
s = sections[['uuid','course_offering_uuid','number']]

#filter Satisfactory or Unsatisfactory
su_count = grades['s_count'] + grades['u_count']
su_filter = (su_count == 0)

#filter Credit or No Credit
crn_count = grades['cr_count'] + grades['n_count']
crn_filter = (crn_count == 0)

#merge grades and sections to add the section id into the grades df
df = pd.merge(g[su_filter & crn_filter], s,
              left_on=['course_offering_uuid','section_number'],
              right_on=['course_offering_uuid','number'],how='left')\
       .drop('number',axis=1)

#rename the section id column
df.rename(columns = {'uuid':'section_uuid'}, inplace=True)


#Information about grades dataset
print('Information about grades dataset:')

#how many duplicates?
print(f'There is {df.shape[0] - g.shape[0]} duplicates.')

#remove duplicates
df.drop_duplicates(['course_offering_uuid','section_number',
                    'a_count','class_size'],inplace=True)

#how many missing grades?
print('There is {} missing grade values.'\
      .format(df[df.class_size==0].shape[0]))

#how many S/U and Cr/N?
print('There is {} S/U and {} Cr/N grade values.'\
      .format((~su_filter).sum(),(~crn_filter).sum()))

#how many missing section ids?
print(f'There is {df.section_uuid.isna().sum()} missing section ids.')

#add column with fraction of A grades
df['a_frac'] = df.a_count / df.class_size

#drop missing section ids and missing grades
df.dropna(axis=0,inplace=True)

print(f'Initaial grades dataset size is {g.shape[0]}, after filtering it is {df.shape[0]}.')

Information about grades dataset:
There is 5595 duplicates.
There is 101050 missing grade values.
There is 16727 S/U and 550 Cr/N grade values.
There is 2697 missing section ids.
Initaial grades dataset size is 193262, after filtering it is 74867.


In [168]:
#####process instructors#####

#merge teachings and instructors dfs to add instructor names
teachers = pd.merge(teachings,instructors,
                    right_on='id',left_on='instructor_id')\
             .drop('id',axis=1)

#rename the section id column
teachers.rename(columns = {'name':'instructor_name'}, inplace=True)

#using info about section id, merge grades and teachers 
df = pd.merge(df,teachers,on = 'section_uuid',how='left')

#how many missing instructor ids?
print(f'There is {df.instructor_id.isna().sum()} missing instructor ids.')

#drop 29 missing instructor ids
df.dropna(axis=0,inplace=True)

#get back int type for the instructor ids
df.instructor_id = df.instructor_id.astype('int64')

#Information about grades dataset
print('Information about instructors dataset:')

print(f'After removing the missing instructor ids dataset size is {df.shape[0]}')

There is 14 missing instructor ids.
Information about instructors dataset:
After removing the missing instructor ids dataset size is 74853


In [169]:
#####process subjects#####

#filter letters from subject codes
subjects = subjects[~(subjects.code == 'ZZZ') & ~(subjects.code == 'SAB')]

#change code type to integer
subjects.code = subjects.code.astype('int64')

#merge subjects and is_stem 
subjects_stem = pd.merge(subjects,is_stem,on='name')

#rename subject column
subjects_stem.rename(columns={'name':'subject_name'},inplace=True)

#merge subject_mem and subjects on code
subject_code = pd.merge(subject_mem,subjects_stem,left_on='subject_code',
                        right_on = 'code',how='left').drop('code',axis=1)

#using info about course_offering_uuid id merge grades and subjects 
df = pd.merge(df,subject_code,on = 'course_offering_uuid',how='left')

#how many multi code courses are there? Keep the first one.
d = subject_code.groupby('course_offering_uuid')['subject_code'].count()
print('There is {} multi code courses.'\
      .format(d[d > 1].index.shape[0]))

#remove duplicates
df.drop_duplicates(['course_offering_uuid','section_number',
                    'a_count','class_size'],inplace=True)

print(f'Current dataset size is {df.shape[0]}.')

There is 9828 multi code courses.
Current dataset size is 74853.


In [170]:
#define class size category
#define bins
bins = df.class_size.quantile([0,0.25,0.75,1]).values
#shift initial value to 0 to avoid nans
bins[0] = 0

df['class_cat'] = pd.cut(df.class_size,
                         bins= bins,
                         labels=['small','medium','large'])

In [171]:
#reorder the df and skip a_count, subject abbreviation
df = df[['course_offering_uuid','section_uuid','section_number',
         'instructor_id','instructor_name','subject_code',
         'subject_name','is_stem','class_size','class_cat','a_frac']]

print(df.shape)
df.info()

(74853, 11)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 74853 entries, 0 to 86451
Data columns (total 11 columns):
course_offering_uuid    74853 non-null object
section_uuid            74853 non-null object
section_number          74853 non-null int64
instructor_id           74853 non-null int64
instructor_name         74853 non-null object
subject_code            74853 non-null int64
subject_name            74853 non-null object
is_stem                 74853 non-null int64
class_size              74853 non-null int64
class_cat               74853 non-null category
a_frac                  74853 non-null float64
dtypes: category(1), float64(1), int64(5), object(4)
memory usage: 6.4+ MB


In [174]:
#write the process dataset
df.to_csv('grades_processed.csv')

In [175]:
grades.describe(include=['object'])

,course_offering_uuid
count,193262
unique,80171
top,36754980-48d2-3b34-ad0c-d621a6cb30df
freq,60


In [179]:
instructors.describe(include=['object']).rename(columns={'name':'instructor_name'})

,instructor_name
count,18736
unique,18598
top,SUSAN SMITH
freq,3


In [186]:
subjects.describe(include=['object']).rename(columns={'name':'subject_name'})[['subject_name']]

,subject_name
count,198
unique,198
top,WILDLIFE ECOLOGY
freq,1


In [6]:
course_offer.describe(include=['object'])

,uuid,course_uuid,name
count,81452,81452,78916
unique,81452,9306,8242
top,e6b8f65b-557e-307b-a1cf-3c07b100304d,aff38538-283c-36f3-bb56-50d75f483e01,Directed Study
freq,1,22,1666


In [7]:
#How many terms are there?
course_offer.term_code.nunique()

22